# ホログラム画像作成

In [ ]:
import math

x_num = 30 # 
y_num = 30 # 
r = 1.5    # 球の半径（mm）
d = 2 * r  # 球の直径でもあり、  広い方向のspace ( in mm)
narrow_space = math.sqrt(3)*r # 狭い方向のspace ( in mm)

# ピクセル数を計算する（プリント時には、現物合わせで変倍する）
pixels_x = int(narrow_space * x_num / 25.4 * 600)  # in pixel
pixels_y = int(d * y_num / 25.4 * 600)             # in pixel
# セルあたりピクセル数を計算する
pixels_of_cell_in_y = int(pixels_y / y_num)
pixels_of_cell_in_x = int(pixels_x / x_num)

import numpy as np
import cv2

# 変角画像を読み込む
files = ["001.png","002.png","003.png","004.png","005.png",
         "006.png","007.png","008.png","009.png"]
images = [cv2.resize(cv2.imread("001.png"), dsize=(x_num, y_num)) for f in files]

import matplotlib.pyplot as plt

# 変角画像を表示
for image in images:
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()

# テクスチャ画像バッファを生成する
background_image2 = np.zeros([pixels_y, pixels_x, 3], dtype = 'uint8')

# テクスチャ画像バッファに、レンズアレイ毎に「変角画像」を貼り付ける
for y in range(y_num):
    for x in range(x_num):
        # 張り込む「変角」画像を生成する
        patch = np.zeros( [ round(pixels_of_cell_in_y),
                            round(pixels_of_cell_in_x), 3], 
                           dtype = 'uint8')
        for y1 in range(3):
            for x1 in range(3):
                patch[ 
            round(y1*pixels_of_cell_in_y/3):round((y1+1)*pixels_of_cell_in_y/3),
            round(x1*pixels_of_cell_in_x/3):round((x1+1)*pixels_of_cell_in_x/3)] = \
                images[3*y1+x1][y,x]
        # 「変角」画像をテクスチャ画像バッファに貼り付ける
        sy = round(y*pixels_of_cell_in_y)
        if y%2 is 0: # 14列の行
            if x < (x_num-1):
                sx = round(x*pixels_of_cell_in_x+pixels_of_cell_in_x/2)
                background_image2[sy:round(sy+pixels_of_cell_in_y), sx: \
                                  round(sx+pixels_of_cell_in_x)]= patch
        else:        # 15列の行
            sx = round(x*pixels_of_cell_in_x)
            background_image2[sy:round(sy+pixels_of_cell_in_y), \ 
                              sx:round(sx+pixels_of_cell_in_x)] = patch
# テクスチャ画像を保存する
cv2.imwrite('patch_30x30.png', background_image2)
# テクスチャ画像を表示
plt.figure(figsize=(4.217,4.217))
plt.imshow(cv2.cvtColor(background_image2, cv2.COLOR_BGR2RGB))